In [1]:
import os
os.getcwd()

'/home/gaowei/.cache/bazel/_bazel_gaowei/468ff0ee32e8cf69d70c757ab23dfef8/execroot/com_nvidia_isaac/bazel-out/k8-opt/bin/ipython_spot/parse_data.runfiles/com_nvidia_isaac/ipython_spot'

In [2]:
#go back to library path.
os.chdir("..")
os.getcwd()

'/home/gaowei/.cache/bazel/_bazel_gaowei/468ff0ee32e8cf69d70c757ab23dfef8/execroot/com_nvidia_isaac/bazel-out/k8-opt/bin/ipython_spot/parse_data.runfiles/com_nvidia_isaac'

In [3]:
# A simple generator wrapper, not sure if it's good for anything at all.
# With basic python threading
from threading import Thread

try:
    from queue import Queue

except ImportError:
    from Queue import Queue
 
# ... or use multiprocessing versions
# WARNING: use sentinel based on value, not identity
from multiprocessing import Process, Queue as MpQueue


class ThreadedGenerator(object):
    """
    Generator that runs on a separate thread, returning values to calling
    thread. Care must be taken that the iterator does not mutate any shared
    variables referenced in the calling thread.
    """

    def __init__(self, iterator,
                 sentinel=object(),
                 queue_maxsize=0,
                 daemon=False,
                 Thread=Thread,
                 Queue=Queue):
        self._iterator = iterator
        self._sentinel = sentinel
        self._queue = Queue(maxsize=queue_maxsize)
        self._thread = Thread(
            name=repr(iterator),
            target=self._run
        )
        self._thread.daemon = daemon

    def __repr__(self):
        return 'ThreadedGenerator({!r})'.format(self._iterator)

    def _run(self):
        try:
            for value in self._iterator:
                self._queue.put(value)

        finally:
            self._queue.put(self._sentinel)

    def __iter__(self):
        self._thread.start()
        for value in iter(self._queue.get, self._sentinel):
            yield value

        self._thread.join()

In [4]:
CASK_FOLDER="/data/spot_data/spot"
TICK_CHANNEL="mid_color"
CHUNK_SIZE=1

from engine.pyalice import Cask, Message
import io
import numpy as np
from PIL import Image
import glob
from tqdm import tqdm
from toolz import partition_all
from itertools import chain
from munch import Munch
import shutil
import os
import os.path as osp
import csv

intentions = {
    0 : "forward",
    1 : "left",
    2 : "right",
    3 : "unknown"
}

def frame_span(cask_path):
    cask = Cask(cask_path)
    tick_channel = None
    for channel in cask.channels:
        if channel.name == TICK_CHANNEL:
            tick_channel = channel
            break
        
    span = 0
    if (tick_channel is not None):
        span = len(tick_channel)
    return span

def debug_data(cask_path, offset):
    print (offset)
    cask = Cask(cask_path)
    # List all channels recorded
    pubtimes=[]
    for channel in cask.channels:
        if channel.name == TICK_CHANNEL:
            for (idx, msg) in enumerate(tqdm(channel)):
                pubtimes.append(idx+offset)
    return pubtimes

def parse_data(cask_path, offset):
    cask = Cask(cask_path)
    channel_sync = {}
    channel_byname = {}
    for channel in cask.channels:
        print (channel.name)
        channel_sync[channel.name] = 0
        channel_byname[channel.name] = channel
        
#     frames = []
    for idx, msg in enumerate(tqdm(channel_byname[TICK_CHANNEL])):
        #sync all other msgs by pubtime
        for key in channel_sync.keys():
            key_idx = channel_sync[key]
            while channel_byname[key][key_idx].pubtime < msg.pubtime:
                key_idx += 1
                if key_idx > len(channel_byname[key])-1:
                    key_idx -= 1
                    break
            channel_sync[key] = key_idx
            
        # check valid data
        left_color = channel_byname['left_color'][channel_sync['left_color']].tensor.T
        mid_color = channel_byname['mid_color'][channel_sync['mid_color']].tensor.T
        right_color = channel_byname['right_color'][channel_sync['right_color']].tensor.T
        if len(left_color) > 0 and len(mid_color) > 0 and len(right_color) > 0:
            #img
            yield Munch(frame=idx+offset,
                            left_color=left_color[0],
                            mid_color=mid_color[0],
                            right_color=right_color[0],
                            intention_control=channel_byname['intention_control'][channel_sync['intention_control']].proto,
                       )

if __name__ == "__main__":
    data_dir = "/home/gaowei/debug"
    
    dirs = [name[:-3] for name in (glob.glob(f"{CASK_FOLDER}/*/*/*kv", recursive=True))]
    dir_span = [frame_span(fn) for fn in dirs]

    cum_sum = [0,] + list(np.cumsum(dir_span))

    results = chain(*[parse_data(fn, cum_sum[idx]) for idx, fn in enumerate(dirs)])
    it = ThreadedGenerator(results, queue_maxsize=6500)

    # make dirs for images
    data_dir = osp.join(data_dir, 'data')
    if os.path.exists(data_dir):
        shutil.rmtree(data_dir)
    os.makedirs(osp.join(data_dir))
    os.makedirs(osp.join(data_dir, 'left_color'))
    os.makedirs(osp.join(data_dir, 'mid_color'))
    os.makedirs(osp.join(data_dir, 'right_color'))
    f = open(osp.join(data_dir, 'label.txt'), 'w')
    
    def write_img(fn, data):
        fObj = open(osp.join(data_dir, fn), "wb")
        fObj.write(data)
        fObj.close()
        
    # write parallelly
    labelwriter = csv.writer(f, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    labelwriter.writerow(['frame', 'intention_type', 'current_velocity', 'steering_wheel_angle', 'dlm'])
    for chunk in partition_all(CHUNK_SIZE, tqdm(it)):
        for c in chunk:
            write_img(osp.join(f"left_color/{c.frame}.jpg"), c.left_color)
            write_img(osp.join(f"mid_color/{c.frame}.jpg"), c.mid_color)
            write_img(osp.join(f"right_color/{c.frame}.jpg"), c.right_color)
            labelwriter.writerow([c.frame, "dlm", c.intention_control.x, c.intention_control.y, intentions[int(c.intention_control.z)]])

0it [00:00, ?it/s]
1it [00:00,  2.99it/s]77 [00:00<?, ?it/s]
86it [00:00,  4.26it/s]377 [00:00<00:03, 5147.83it/s]

right_depth
left_depth
mid_depth
state
intention_control
spot_odom
right_color
mid_color
left_color



166it [00:00,  6.07it/s]77 [00:00<00:03, 4923.69it/s]
244it [00:00,  8.65it/s]377 [00:00<00:03, 4836.87it/s]
336it [00:00, 12.30it/s]377 [00:00<00:03, 4725.15it/s]
405it [00:00, 17.44it/s]377 [00:00<00:03, 4877.08it/s]
473it [00:00, 24.64it/s]377 [00:00<00:03, 4959.47it/s]
539it [00:01, 34.59it/s]377 [00:00<00:03, 5124.15it/s]
604it [00:01, 48.19it/s]377 [00:00<00:03, 5188.20it/s]
667it [00:01, 66.59it/s]377 [00:00<00:03, 5176.76it/s]
757it [00:01, 92.17it/s]377 [00:01<00:03, 4803.91it/s]
855it [00:01, 126.56it/s]77 [00:01<00:03, 4746.15it/s]
961it [00:01, 171.95it/s]77 [00:01<00:03, 4645.90it/s]
1049it [00:01, 226.65it/s]7 [00:01<00:03, 4587.06it/s]
1152it [00:01, 295.78it/s]7 [00:01<00:02, 4585.23it/s]
1251it [00:01, 374.50it/s]7 [00:01<00:02, 4560.99it/s]
1651it [00:02, 617.72it/s]7 [00:01<00:02, 4530.01it/s]
1963it [00:02, 813.33it/s]7 [00:01<00:03, 4030.24it/s]
2273it [00:02, 1044.25it/s] [00:01<00:03, 3708.97it/s]
2895it [00:02, 1579.46it/s] [00:02<00:03, 3506.41it/s]
3213it [00

right_depth
mid_color
left_color
right_color
intention_control
state
spot_odom
left_depth
mid_depth



14675it [00:06, 2659.88it/s]0:00<00:00, 3381.43it/s]
15019it [00:06, 2853.33it/s]00:00<00:00, 3380.68it/s]
15363it [00:06, 3007.15it/s]00:00<00:00, 3421.45it/s]
15689it [00:06, 2957.83it/s]00:00<00:00, 3289.44it/s]
16003it [00:06, 2921.89it/s]00:00<00:00, 3138.46it/s]
16308it [00:06, 2912.89it/s]00:00<00:00, 3042.36it/s]
16608it [00:06, 2892.60it/s]00:00<00:00, 2990.34it/s]
16904it [00:07, 2862.92it/s]00:00<00:00, 2937.26it/s]
17220it [00:07, 2944.44it/s]00:01<00:00, 2898.60it/s]
17525it [00:07, 2975.26it/s]00:01<00:00, 2983.79it/s]
100%|██████████| 3949/3949 [00:01<00:00, 3090.08it/s]
17826it [00:07, 2556.86it/s]
18094it [00:07, 1234.77it/s]:00<?, ?it/s]
18432it [00:07, 1525.17it/s]00:00<00:08, 3402.64it/s]

...


55605it [00:19, 1902.01it/s]00:00<00:07, 3303.73it/s]
55955it [00:19, 2202.93it/s][00:00<00:06, 3360.88it/s]
56288it [00:19, 2451.43it/s][00:00<00:06, 3395.26it/s]
56602it [00:19, 2578.92it/s][00:00<00:07, 3228.94it/s]
56910it [00:19, 2685.74it/s][00:00<00:07, 3157.35it/s]
57215it [00:19, 2759.59it/s][00:00<00:07, 3084.93it/s]
57517it [00:20, 2411.45it/s][00:00<00:08, 2556.17it/s]
57813it [00:20, 2551.98it/s][00:00<00:08, 2662.34it/s]
58108it [00:20, 2657.53it/s][00:01<00:07, 2741.72it/s]
58396it [00:20, 2718.33it/s][00:01<00:07, 2793.21it/s]
58685it [00:20, 2766.64it/s][00:01<00:07, 2795.78it/s]
58978it [00:20, 2811.49it/s][00:01<00:07, 2861.79it/s]
59265it [00:20, 2800.08it/s][00:01<00:07, 2832.58it/s]
59550it [00:20, 2538.91it/s][00:01<00:07, 2799.58it/s]
 26%|██▌       | 6267/24423 [00:01<00:04, 3813.22it/s]
60060it [00:21, 1498.85it/s][00:02<00:04, 3968.47it/s]
60322it [00:21, 1717.97it/s][00:02<00:04, 3550.49it/s]
60589it [00:21, 1922.43it/s][00:02<00:05, 3209.52it/s]
60858it [0

left_color
right_depth
mid_color
intention_control
state
mid_depth
spot_odom
left_depth
right_color



72533it [00:27, 1946.77it/s]0:00<00:03, 2599.18it/s]
72755it [00:27, 2013.87it/s]0:00<00:03, 2585.65it/s]
72976it [00:27, 1989.64it/s]0:00<00:03, 2395.39it/s]
73226it [00:27, 2118.28it/s]00:00<00:03, 2334.40it/s]
73455it [00:28, 2164.02it/s]00:00<00:03, 2392.57it/s]
73680it [00:28, 2183.67it/s]00:00<00:03, 2442.69it/s]
73912it [00:28, 2221.36it/s]00:00<00:03, 2392.74it/s]
74168it [00:28, 2311.55it/s]00:00<00:03, 2373.21it/s]
74404it [00:28, 2210.88it/s]00:01<00:02, 2426.89it/s]
 29%|██▉       | 2757/9557 [00:01<00:02, 2601.36it/s]
74629it [00:28, 1997.08it/s]00:01<00:02, 2822.86it/s]
74867it [00:28, 2097.00it/s]00:01<00:02, 2734.37it/s]
75086it [00:28, 2124.00it/s]00:01<00:02, 2779.09it/s]
75342it [00:28, 2237.00it/s]00:01<00:02, 2706.08it/s]
75575it [00:28, 2261.19it/s]00:01<00:01, 2704.29it/s]
75805it [00:29, 2241.26it/s]00:01<00:01, 2729.33it/s]
76032it [00:29, 2001.61it/s]00:01<00:01, 2847.17it/s]
 54%|█████▍    | 5181/9557 [00:01<00:01, 2997.04it/s]
76497it [00:29, 1865.43it/s]00

mid_color
intention_control
right_color
mid_depth
left_depth
spot_odom
state
right_depth
left_color



80859it [00:31, 1759.17it/s]00:00<00:08, 2816.70it/s]
81139it [00:31, 1979.07it/s]00:00<00:08, 2810.12it/s]
81429it [00:31, 2187.34it/s][00:00<00:08, 2822.79it/s]
81714it [00:31, 2349.59it/s][00:00<00:08, 2841.04it/s]
82001it [00:32, 2483.09it/s][00:00<00:07, 2843.18it/s]
82277it [00:32, 2548.09it/s][00:00<00:08, 2796.67it/s]
82552it [00:32, 2591.26it/s][00:00<00:08, 2770.02it/s]
82826it [00:32, 2587.28it/s][00:00<00:08, 2703.92it/s]
83128it [00:32, 2703.31it/s][00:01<00:07, 2754.16it/s]
83448it [00:32, 2835.26it/s][00:01<00:07, 2859.11it/s]
83769it [00:32, 2936.07it/s][00:01<00:07, 2933.18it/s]
84105it [00:32, 3050.58it/s][00:01<00:06, 3022.37it/s]
84437it [00:32, 3125.56it/s][00:01<00:06, 3121.22it/s]
84772it [00:32, 3187.53it/s][00:01<00:06, 3181.93it/s]
85106it [00:33, 3230.15it/s][00:01<00:06, 3239.19it/s]
85448it [00:33, 3282.30it/s][00:01<00:05, 3269.17it/s]
85793it [00:33, 3330.20it/s][00:01<00:05, 3317.37it/s]
86128it [00:33, 3288.38it/s][00:01<00:05, 3323.85it/s]
86459it [00

In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

for idx, r in enumerate(results):
    plt.vlines(x=idx, ymin=r[0], ymax=r[-1], colors='green', lw=8)
